# Week 6: Data Visualization for Marketing Analytics

**Goal:** Master data visualization to communicate marketing insights effectively.

**Time Commitment:** ~1 hour per day × 7 days = 7 hours total

**What You'll Learn:**
- Matplotlib fundamentals and customization
- Seaborn for statistical visualizations
- Time series visualization techniques
- Distribution plots (histograms, box plots, violin plots)
- Heatmaps and correlation matrices
- Dashboard design principles
- Building a marketing performance dashboard

**Why This Matters:**
As a Marketing Measurement Partner, you need to:
- Present insights to non-technical stakeholders
- Create compelling reports and dashboards
- Spot trends and patterns visually
- Make data accessible and actionable

"A picture is worth a thousand rows of data."

The best analysis is useless if you can't communicate it effectively.

---

## Setup: Load Data and Libraries

We'll use the same marketing dataset from Week 5.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Set visualization defaults
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('Set2')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 2)

print("✅ Libraries imported and style configured!")

In [ ]:
# Generate marketing dataset (same as Week 5)
np.random.seed(42)

dates = pd.date_range('2024-10-01', periods=90, freq='D')
campaigns = [
    'Search_Brand', 'Search_Generic', 'FB_Prospecting', 'FB_Retargeting',
    'Display_Remarketing', 'Instagram_Stories', 'LinkedIn_B2B', 
    'TikTok_Awareness', 'YouTube_Video', 'Search_Competitor'
]
channels = ['Google', 'Google', 'Meta', 'Meta', 'Google', 'Meta', 'LinkedIn', 'TikTok', 'Google', 'Google']
campaign_types = ['Search', 'Search', 'Social', 'Social', 'Display', 'Social', 'Social', 'Video', 'Video', 'Search']

data = []
for i, campaign in enumerate(campaigns):
    for date in dates:
        dow_multiplier = 1.3 if date.dayofweek < 5 else 0.7
        
        if campaign_types[i] == 'Search':
            impressions = int(np.random.normal(5000, 1000) * dow_multiplier)
            ctr = np.random.normal(0.05, 0.01)
            cvr = np.random.normal(0.04, 0.008)
        elif campaign_types[i] == 'Social':
            impressions = int(np.random.normal(25000, 5000) * dow_multiplier)
            ctr = np.random.normal(0.03, 0.008)
            cvr = np.random.normal(0.025, 0.006)
        elif campaign_types[i] == 'Display':
            impressions = int(np.random.normal(40000, 8000) * dow_multiplier)
            ctr = np.random.normal(0.015, 0.004)
            cvr = np.random.normal(0.02, 0.005)
        else:  # Video
            impressions = int(np.random.normal(50000, 10000) * dow_multiplier)
            ctr = np.random.normal(0.02, 0.005)
            cvr = np.random.normal(0.015, 0.004)
        
        clicks = int(impressions * max(0.001, ctr))
        conversions = int(clicks * max(0.001, cvr))
        cpc = np.random.uniform(0.5, 3.0) if campaign_types[i] == 'Search' else np.random.uniform(0.3, 1.5)
        cost = clicks * cpc
        aov = np.random.uniform(80, 200)
        revenue = conversions * aov
        
        data.append({
            'date': date,
            'campaign': campaign,
            'channel': channels[i],
            'campaign_type': campaign_types[i],
            'impressions': max(0, impressions),
            'clicks': max(0, clicks),
            'conversions': max(0, conversions),
            'cost': max(0, cost),
            'revenue': max(0, revenue)
        })

df = pd.DataFrame(data)

# Calculate metrics
df['ctr'] = df['clicks'] / df['impressions']
df['cvr'] = df['conversions'] / df['clicks'].replace(0, np.nan)
df['cpa'] = df['cost'] / df['conversions'].replace(0, np.nan)
df['roas'] = df['revenue'] / df['cost'].replace(0, np.nan)
df['cpc'] = df['cost'] / df['clicks'].replace(0, np.nan)

print(f"✅ Dataset loaded: {len(df)} rows, {len(campaigns)} campaigns, {len(dates)} days")

## 📅 Day 36: Matplotlib Basics (~60 min)

### Learning Objectives
- Understand matplotlib figure and axes structure
- Create line plots, bar charts, and scatter plots
- Customize colors, labels, titles, and legends
- Create multi-panel figures

### The Business Problem
You need to visualize:
- Campaign performance over time
- Channel comparisons
- Metric relationships
- Trends and patterns

### 📖 Concept: Basic Line Plot

Line plots show trends over time.

In [ ]:
# Simple line plot - daily conversions for one campaign
campaign_data = df[df['campaign'] == 'Search_Brand'].sort_values('date')

plt.figure(figsize=(12, 6))
plt.plot(campaign_data['date'], campaign_data['conversions'])
plt.title('Search Brand Campaign - Daily Conversions', fontsize=14, fontweight='bold')
plt.xlabel('Date')
plt.ylabel('Conversions')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### 📖 Concept: Multiple Lines

Compare multiple series on one plot.

In [ ]:
# Compare top 3 campaigns
top_campaigns = ['Search_Brand', 'FB_Prospecting', 'Search_Generic']

plt.figure(figsize=(12, 6))
for campaign in top_campaigns:
    campaign_data = df[df['campaign'] == campaign].sort_values('date')
    plt.plot(campaign_data['date'], campaign_data['conversions'], label=campaign, linewidth=2)

plt.title('Daily Conversions - Top 3 Campaigns', fontsize=14, fontweight='bold')
plt.xlabel('Date')
plt.ylabel('Conversions')
plt.legend(loc='best')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### 📖 Concept: Bar Charts

Bar charts compare categories.

In [ ]:
# Total conversions by campaign
campaign_totals = df.groupby('campaign')['conversions'].sum().sort_values(ascending=False)

plt.figure(figsize=(12, 6))
plt.bar(range(len(campaign_totals)), campaign_totals.values, color='steelblue', edgecolor='black')
plt.xticks(range(len(campaign_totals)), campaign_totals.index, rotation=45, ha='right')
plt.title('Total Conversions by Campaign', fontsize=14, fontweight='bold')
plt.xlabel('Campaign')
plt.ylabel('Total Conversions')
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

### 📖 Concept: Scatter Plots

Scatter plots show relationships between two variables.

In [ ]:
# Cost vs Revenue relationship
plt.figure(figsize=(10, 6))
plt.scatter(df['cost'], df['revenue'], alpha=0.5, s=30)
plt.title('Daily Cost vs Revenue', fontsize=14, fontweight='bold')
plt.xlabel('Cost ($)')
plt.ylabel('Revenue ($)')
plt.grid(True, alpha=0.3)

# Add reference line (ROAS = 1.0)
max_val = max(df['cost'].max(), df['revenue'].max())
plt.plot([0, max_val], [0, max_val], 'r--', label='ROAS = 1.0', linewidth=2)
plt.legend()
plt.tight_layout()
plt.show()

### 📖 Concept: Subplots

Create multiple charts in one figure.

In [ ]:
# 2x2 grid of plots
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Plot 1: Total spend by channel
channel_spend = df.groupby('channel')['cost'].sum()
axes[0, 0].bar(channel_spend.index, channel_spend.values, color='coral')
axes[0, 0].set_title('Total Spend by Channel', fontweight='bold')
axes[0, 0].set_ylabel('Cost ($)')

# Plot 2: Total conversions by channel
channel_conv = df.groupby('channel')['conversions'].sum()
axes[0, 1].bar(channel_conv.index, channel_conv.values, color='steelblue')
axes[0, 1].set_title('Total Conversions by Channel', fontweight='bold')
axes[0, 1].set_ylabel('Conversions')

# Plot 3: Daily trend for one campaign
campaign_data = df[df['campaign'] == 'Search_Brand'].sort_values('date')
axes[1, 0].plot(campaign_data['date'], campaign_data['roas'], color='green', linewidth=2)
axes[1, 0].set_title('Search Brand - Daily ROAS', fontweight='bold')
axes[1, 0].set_xlabel('Date')
axes[1, 0].set_ylabel('ROAS')
axes[1, 0].tick_params(axis='x', rotation=45)

# Plot 4: CTR distribution
axes[1, 1].hist(df['ctr'], bins=30, color='purple', alpha=0.7, edgecolor='black')
axes[1, 1].set_title('CTR Distribution', fontweight='bold')
axes[1, 1].set_xlabel('CTR')
axes[1, 1].set_ylabel('Frequency')

plt.tight_layout()
plt.show()

### ✏️ Exercise 1: Create Custom Visualizations

In [ ]:
# YOUR CODE HERE
# Create a 1x3 subplot showing:
# 1. Line plot: Weekly average ROAS for all campaigns (aggregate daily to weekly)
# 2. Bar chart: Average CPA by campaign type
# 3. Scatter plot: CTR vs CVR, colored by channel
#
# Make it presentation-ready with proper titles, labels, and legends



### 🎯 Day 36 Mini-Project: Campaign Performance Dashboard

Create a comprehensive 6-panel dashboard using matplotlib.

In [ ]:
# YOUR CODE HERE
# Create a 2x3 dashboard with:
# 1. Daily total conversions (all campaigns combined)
# 2. Cumulative spend over time
# 3. ROAS by campaign (horizontal bar chart, sorted)
# 4. Daily cost by channel (stacked area or line)
# 5. Cost vs Revenue scatter (colored by campaign type)
# 6. Conversion distribution (histogram)
#
# Add a main title for the entire dashboard
# Use consistent color scheme
# Make it executive-ready!



### 🎓 Day 36 Key Takeaways

✅ Line plots show trends over time  
✅ Bar charts compare categories  
✅ Scatter plots reveal relationships  
✅ Subplots enable multi-view analysis  
✅ Customization makes charts professional  

**Next:** Tomorrow we'll use Seaborn for statistical visualizations!

---

## 📅 Day 37: Seaborn for Statistical Plots (~60 min)

### Learning Objectives
- Use Seaborn for statistical visualizations
- Create box plots and violin plots
- Build categorical plots (bar, point, count)
- Use FacetGrid for multi-dimensional views

### The Business Problem
Seaborn makes statistical visualization easier and more beautiful:
- Compare distributions across segments
- Visualize relationships with confidence intervals
- Create publication-ready statistical graphics

### 📖 Concept: Box Plots

Box plots show distribution summary (quartiles, median, outliers).

In [ ]:
# ROAS distribution by channel
plt.figure(figsize=(10, 6))
sns.boxplot(data=df, x='channel', y='roas', palette='Set2')
plt.title('ROAS Distribution by Channel', fontsize=14, fontweight='bold')
plt.ylabel('ROAS')
plt.xlabel('Channel')
plt.tight_layout()
plt.show()

### 📖 Concept: Violin Plots

Violin plots combine box plot with distribution density.

In [ ]:
# CPA distribution by campaign type
plt.figure(figsize=(10, 6))
sns.violinplot(data=df, x='campaign_type', y='cpa', palette='muted')
plt.title('CPA Distribution by Campaign Type', fontsize=14, fontweight='bold')
plt.ylabel('CPA ($)')
plt.xlabel('Campaign Type')
plt.ylim(0, 100)  # Limit y-axis for readability
plt.tight_layout()
plt.show()

### 📖 Concept: Bar Plots with Error Bars

Seaborn automatically adds confidence intervals.

In [ ]:
# Average conversions by channel with confidence intervals
plt.figure(figsize=(10, 6))
sns.barplot(data=df, x='channel', y='conversions', errorbar='ci', palette='Set1')
plt.title('Average Daily Conversions by Channel (with 95% CI)', fontsize=14, fontweight='bold')
plt.ylabel('Average Conversions')
plt.xlabel('Channel')
plt.tight_layout()
plt.show()

### 📖 Concept: Point Plots

Point plots show means and confidence intervals, great for trends.

In [ ]:
# Weekly ROAS trend
df['week'] = df['date'].dt.isocalendar().week

plt.figure(figsize=(12, 6))
sns.pointplot(data=df, x='week', y='roas', hue='channel', errorbar='ci', palette='deep')
plt.title('Weekly ROAS Trend by Channel', fontsize=14, fontweight='bold')
plt.ylabel('ROAS')
plt.xlabel('Week')
plt.legend(title='Channel', loc='best')
plt.tight_layout()
plt.show()

### 📖 Concept: FacetGrid

Create multiple plots split by categorical variable.

In [ ]:
# ROAS distribution for each channel
g = sns.FacetGrid(df, col='channel', col_wrap=2, height=4, aspect=1.5)
g.map(sns.histplot, 'roas', bins=20, kde=True)
g.set_titles('{col_name}', fontweight='bold')
g.set_axis_labels('ROAS', 'Frequency')
g.fig.suptitle('ROAS Distribution by Channel', y=1.02, fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

### ✏️ Exercise 2: Seaborn Statistical Visualizations

In [ ]:
# YOUR CODE HERE
# Create three visualizations:
# 1. Violin plot comparing CVR across campaign types
# 2. Point plot showing daily average cost trend by week and campaign_type
# 3. FacetGrid with scatter plots of cost vs conversions, one panel per channel
#
# Customize colors and labels appropriately



### 🎯 Day 37 Mini-Project: Statistical Comparison Dashboard

Create a comprehensive statistical comparison across segments.

In [ ]:
# YOUR CODE HERE
# Build a 2x2 statistical dashboard:
# 1. Box plot: ROAS by campaign (show all 10 campaigns)
# 2. Violin plot: CPA by channel with overlay showing individual points
# 3. Bar plot: Average daily revenue by campaign type (with error bars)
# 4. Point plot: CTR trend over weeks, separate line for each channel
#
# Use appropriate color palettes
# Add proper titles and labels
# Make insights clear at a glance



### 🎓 Day 37 Key Takeaways

✅ Seaborn simplifies statistical visualization  
✅ Box/violin plots show distribution characteristics  
✅ Error bars communicate uncertainty  
✅ FacetGrid enables small multiples  
✅ Seaborn integrates seamlessly with pandas  

**Next:** Tomorrow we'll focus on time series visualization!

---

## 📅 Day 38-41: Additional Visualization Topics

**Day 38: Time Series Visualization**
- Date formatting and tick control
- Rolling averages overlay
- Seasonal decomposition plots
- Cumulative trend visualization

**Day 39: Distribution Plots**
- Histograms with KDE
- Q-Q plots for normality
- ECDF plots
- Joint plots (bivariate distributions)

**Day 40: Heatmaps and Correlation Plots**
- Correlation heatmaps
- Pivot table heatmaps
- Calendar heatmaps
- Pair plots for multi-variable analysis

**Day 41: Dashboard Design Principles**
- Layout and hierarchy
- Color theory for data
- Chart selection guidelines
- Accessibility considerations

---

## 📅 Day 42: Week 6 Capstone - Marketing Performance Dashboard (~60 min)

### Project: Executive Marketing Performance Dashboard

**Scenario:**  
Create a comprehensive visual dashboard for quarterly business review. The executive team needs to understand:
- Overall portfolio performance
- Channel and campaign effectiveness
- Trends and patterns
- Opportunities and risks

**Requirements:**
- Professional, executive-ready design
- Clear visual hierarchy
- Mix of chart types appropriate for each insight
- Actionable insights visible at a glance

**Deliverables:**
Create a multi-page dashboard with:
1. Executive summary page (4-6 key metrics)
2. Channel performance page
3. Campaign deep-dive page
4. Time-based trends page

### Page 1: Executive Summary

In [ ]:
# YOUR CODE HERE
# Create a 2x3 executive summary dashboard showing:
# 
# 1. Total Spend (text box or single metric)
# 2. Total Revenue (text box or single metric)
# 3. Overall ROAS (text box or single metric)
# 4. Daily spend trend with 7-day moving average
# 5. Channel mix (pie or donut chart of spend %)
# 6. Top 5 campaigns by ROAS (horizontal bar)
#
# Use large fonts for key numbers
# Add annotations for context
# Use professional color scheme



### Page 2: Channel Performance Deep Dive

In [ ]:
# YOUR CODE HERE
# Create a 2x2 channel analysis dashboard:
#
# 1. Stacked bar: Spend and Revenue by channel (side by side)
# 2. Box plot: ROAS distribution by channel
# 3. Line plot: Daily conversions by channel over time
# 4. Scatter: CPA vs Conversion volume by channel (bubble size = spend)
#
# Color-code consistently by channel across all plots
# Add reference lines where appropriate (e.g., target ROAS)



### Page 3: Campaign Effectiveness

In [ ]:
# YOUR CODE HERE
# Create a comprehensive campaign view:
#
# 1. Scatter matrix (cost, conversions, ROAS) colored by campaign type
# 2. Heatmap: Campaign performance by week (rows=campaigns, cols=weeks, value=ROAS)
# 3. Violin plots: CTR and CVR distributions by campaign
# 4. Bar chart: Campaigns ranked by efficiency score (composite metric)
#
# Highlight top and bottom performers
# Use diverging colormap for heatmap



### Page 4: Trends and Patterns

In [ ]:
# YOUR CODE HERE
# Create a trends dashboard:
#
# 1. Time series: Daily ROAS with 7-day and 14-day moving averages
# 2. Calendar heatmap: Conversions by day of week and week number
# 3. Line plot: Cumulative spend vs cumulative revenue (budget pacing)
# 4. Area chart: Daily cost breakdown by channel (stacked)
#
# Show seasonality and trends clearly
# Highlight anomalies or unusual days



### Bonus: Interactive Dashboard Elements

In [ ]:
# YOUR CODE HERE (Optional)
# Add interactive elements:
#
# 1. Create a function that generates dashboard for any date range
# 2. Add filters for channel, campaign, or time period
# 3. Create drill-down capability (click channel → see campaigns)
#
# This is advanced but valuable for real-world use!



### Dashboard Summary and Insights

In [ ]:
# YOUR CODE HERE
# Write a narrative summary based on your visualizations:
#
# 1. Key findings (3-5 bullets)
# 2. Performance highlights
# 3. Areas of concern
# 4. Specific recommendations (with supporting data from visuals)
# 5. Next steps
#
# Remember: Dashboards should tell a story!

print("""
EXECUTIVE SUMMARY
================

Key Findings:
- [Your insight here based on visualizations]
- [Your insight here]
- [Your insight here]

Recommendations:
1. [Actionable recommendation with data support]
2. [Actionable recommendation]
3. [Actionable recommendation]

Next Steps:
- [Specific action item]
- [Specific action item]
""")

### 🎓 Week 6 Complete!

**Congratulations!** You've completed Week 6 of the Marketing Measurement Partner Academy.

**What You've Mastered:**
- ✅ Matplotlib fundamentals (line, bar, scatter, subplots)
- ✅ Seaborn for statistical visualizations
- ✅ Time series visualization techniques
- ✅ Distribution and relationship plots
- ✅ Heatmaps and correlation matrices
- ✅ Dashboard design principles
- ✅ Building executive-ready dashboards

**Your Visualization Toolkit:**
You can now:
- Choose the right chart for any insight
- Create professional, publication-ready visuals
- Build comprehensive dashboards
- Communicate complex data simply
- Tell compelling data stories

**Next Week Preview:**
Week 7 will cover **Statistics Foundations**:
- Probability basics
- Confidence intervals
- Hypothesis testing
- T-tests and chi-square tests
- Correlation vs causation
- Statistical analysis of campaigns

**You're becoming a visual storyteller with data!** 🚀

---